# Solutions Engineering Workload Modeling

## Project Overview

This Jupyter notebook models Solutions Engineering workload based on revenue targets, conversion rates, staffing levels, and activity assumptions. It enables scenario modeling to understand capacity requirements and utilization across different business conditions.

### Key Capabilities
- **Revenue-Driven Pipeline Modeling**: Calculate required activities from quarterly revenue goals
- **Interactive Parameter Controls**: Adjust assumptions and see real-time impact
- **Capacity Analysis**: Understand SE team utilization and identify constraints
- **Account Management Modeling**: Factor in existing customer meeting requirements
- **Strategic Activity Planning**: Allocate time for non-pipeline activities
- **Professional Visualizations**: Executive-ready charts and dashboards

### Business Applications
- Staffing planning and capacity forecasting
- Revenue target feasibility analysis
- Resource allocation optimization
- Scenario planning for different business conditions

## Introduction

### Workload Modeling Approach

This model uses a **revenue-driven approach** to calculate SE workload requirements:

1. **Start with Revenue Goals**: Quarterly revenue targets drive required new logo counts
2. **Work Backwards Through Funnel**: Calculate required demos, evaluations, and meetings
3. **Factor in Activity Types**: Different evaluation types require different SE time investments
4. **Add Account Management**: Existing customer meetings and onboarding requirements
5. **Include Strategic Activities**: Time for customer zero, content creation, and enablement
6. **Calculate Utilization**: Compare total workload to available SE capacity

### Key Assumptions
- **40-hour work week** standard capacity per SE
- **Conversion rates** remain consistent across evaluation types
- **Account meeting frequencies** based on customer segment strategy
- **Strategic activities** distributed between ICs and Directors based on role allocation

## Library Imports and Configuration

In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive widgets for parameter controls
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configure matplotlib for inline display
%matplotlib inline

# Set seaborn style for professional appearance
sns.set_style("whitegrid")
sns.set_palette("husl")

# Configure pandas display options for better table formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 2)

# Configure matplotlib for better chart appearance
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11

print("✅ All libraries imported successfully")
print("✅ Configuration applied")
print("📊 Ready to begin workload modeling")

## Table of Contents

### 📋 Core Sections
1. [**Helper Functions Framework**](#helper-functions) - Core calculation functions
2. [**Parameter Management**](#parameter-management) - Input parameter definitions and validation
3. [**Revenue & Pipeline Calculations**](#revenue-pipeline) - Core funnel math and workload calculations
4. [**Account Management Logic**](#account-management) - Meeting frequency and time allocation
5. [**Strategic Activities Logic**](#strategic-activities) - Strategic time allocation and distribution

### 🎛️ Interactive Interface
6. [**Interactive Parameter Controls**](#interactive-controls) - Widget-based parameter input
7. [**Real-time Calculations**](#real-time-calculations) - Live calculation updates

### 📊 Visualization & Results
8. [**Core Visualizations**](#core-visualizations) - Essential charts for utilization and capacity
9. [**Executive Dashboard**](#executive-dashboard) - Professional dashboard with multiple chart types
10. [**Results Summary & Export**](#results-summary) - Executive summary and export capabilities

### 🔧 Testing & Validation
11. [**Testing Framework**](#testing) - Comprehensive validation and testing
12. [**Usage Examples**](#examples) - Sample scenarios and use cases

---

# 1. Helper Functions Framework {#helper-functions}

*Core calculation functions that will be reused throughout the notebook*

In [ ]:
# Core helper functions for SE workload modeling calculations

def calculate_required_new_logos(quarterly_revenue_goal, average_selling_price):
    """
    Calculate the number of new logos needed to meet quarterly revenue goals.
    
    Parameters:
    - quarterly_revenue_goal (float): Target revenue for the quarter
    - average_selling_price (float): Average deal size
    
    Returns:
    - float: Number of new logos required
    """
    if average_selling_price <= 0:
        raise ValueError("Average selling price must be greater than 0")
    
    return quarterly_revenue_goal / average_selling_price


def calculate_weighted_win_rate(eval_mix_percentages, win_rates):
    """
    Calculate weighted average win rate across all evaluation types.
    
    Parameters:
    - eval_mix_percentages (dict): Percentages for each evaluation type
    - win_rates (dict): Win rates for each evaluation type
    
    Returns:
    - float: Weighted average win rate (0-1)
    """
    total_percentage = sum(eval_mix_percentages.values())
    if abs(total_percentage - 100) > 0.01:  # Allow small floating point errors
        raise ValueError(f"Evaluation mix percentages must sum to 100%, got {total_percentage}%")
    
    weighted_sum = 0
    for eval_type in eval_mix_percentages:
        if eval_type not in win_rates:
            raise ValueError(f"Win rate not provided for evaluation type: {eval_type}")
        weighted_sum += (eval_mix_percentages[eval_type] / 100) * (win_rates[eval_type] / 100)
    
    return weighted_sum


def calculate_required_activities(new_logos_needed, conversion_rates, eval_mix, win_rates):
    """
    Work backwards through conversion funnel to calculate required weekly activities.
    
    Parameters:
    - new_logos_needed (float): Required new logos per quarter
    - conversion_rates (dict): Conversion rates for each funnel stage
    - eval_mix (dict): Percentage mix of evaluation types
    - win_rates (dict): Win rates by evaluation type
    
    Returns:
    - dict: Required weekly activities by type
    """
    # Calculate weighted win rate
    weighted_win_rate = calculate_weighted_win_rate(eval_mix, win_rates)
    
    # Work backwards through funnel (quarterly to weekly)
    weeks_per_quarter = 13
    
    # Required evaluations per quarter
    required_evaluations_quarterly = new_logos_needed / weighted_win_rate
    
    # Required demos per quarter (from tech eval conversion rate)
    required_demos_quarterly = required_evaluations_quarterly / (conversion_rates['tech_eval_conversion_rate'] / 100)
    
    # Required opportunities per quarter (from demo conversion rate)
    required_opportunities_quarterly = required_demos_quarterly / (conversion_rates['demo_conversion_rate'] / 100)
    
    # Required initial meetings per quarter (from opportunity creation rate)
    required_meetings_quarterly = required_opportunities_quarterly / (conversion_rates['opportunity_creation_rate'] / 100)
    
    # Convert to weekly and break down evaluations by type
    weekly_activities = {
        'initial_meetings': required_meetings_quarterly / weeks_per_quarter,
        'demos': required_demos_quarterly / weeks_per_quarter,
        'total_evaluations': required_evaluations_quarterly / weeks_per_quarter
    }
    
    # Break down evaluations by type
    for eval_type, percentage in eval_mix.items():
        eval_key = f"{eval_type.replace('_percentage', '')}_evaluations"
        weekly_activities[eval_key] = weekly_activities['total_evaluations'] * (percentage / 100)
    
    return weekly_activities


def calculate_evaluation_workload(weekly_evaluations_by_type, time_per_type):
    """
    Calculate SE time requirements for different evaluation types.
    
    Parameters:
    - weekly_evaluations_by_type (dict): Weekly evaluation counts by type
    - time_per_type (dict): Time requirements for each evaluation type
    
    Returns:
    - dict: Weekly hours needed by evaluation type
    """
    evaluation_hours = {}
    
    # Self-guided: ongoing weekly support hours
    if 'self_guided' in weekly_evaluations_by_type:
        evaluation_hours['self_guided'] = (weekly_evaluations_by_type['self_guided'] * 
                                         time_per_type['self_guided_support_hours_per_week'])
    
    # SE-led: ongoing weekly support hours  
    if 'se_led' in weekly_evaluations_by_type:
        evaluation_hours['se_led'] = (weekly_evaluations_by_type['se_led'] * 
                                    time_per_type['se_led_eval_hours_per_week'])
    
    # Rapid POV: total hours distributed over evaluation period (assume 2 weeks)
    if 'rapid_pov' in weekly_evaluations_by_type:
        rapid_pov_weeks = 2  # Standard duration for rapid POV
        evaluation_hours['rapid_pov'] = (weekly_evaluations_by_type['rapid_pov'] * 
                                       time_per_type['rapid_pov_total_hours'] / rapid_pov_weeks)
    
    # No evaluation type requires no SE time
    evaluation_hours['no_eval'] = 0
    
    return evaluation_hours


def calculate_demo_workload(weekly_demos, demo_time_params):
    """
    Calculate total demo workload including prep, delivery, and follow-up.
    
    Parameters:
    - weekly_demos (float): Number of demos per week
    - demo_time_params (dict): Time parameters for demo activities
    
    Returns:
    - float: Total weekly hours for demo activities
    """
    total_demo_time_per_demo = (demo_time_params['demo_prep_time'] + 
                               demo_time_params['demo_delivery_time'] + 
                               demo_time_params['demo_followup_time'])
    
    return weekly_demos * total_demo_time_per_demo


def calculate_meeting_workload(weekly_meetings, meeting_time):
    """
    Calculate workload for initial meetings including prep time.
    
    Parameters:
    - weekly_meetings (float): Number of initial meetings per week
    - meeting_time (float): Time per meeting including prep
    
    Returns:
    - float: Total weekly hours for initial meetings
    """
    return weekly_meetings * meeting_time


def calculate_account_meetings(account_counts, meeting_frequencies):
    """
    Calculate total meetings per month for existing accounts.
    
    Parameters:
    - account_counts (dict): Number of accounts by segment
    - meeting_frequencies (dict): Meeting frequency by segment (meetings/month)
    
    Returns:
    - dict: Monthly meetings by account segment
    """
    monthly_meetings = {}
    
    for segment, count in account_counts.items():
        if segment in meeting_frequencies:
            if segment == 'retain':
                # Retain accounts: frequency is "every X months", so meetings/month = 1/X
                meetings_per_month = count / meeting_frequencies[segment]
            else:
                # Other segments: frequency is meetings per month
                meetings_per_month = count * meeting_frequencies[segment]
            
            monthly_meetings[segment] = meetings_per_month
    
    return monthly_meetings


def calculate_new_logo_onboarding_meetings(new_logos_per_quarter, onboarding_params):
    """
    Calculate new customer onboarding meeting requirements.
    
    Parameters:
    - new_logos_per_quarter (float): New customers per quarter
    - onboarding_params (dict): Onboarding meeting parameters
    
    Returns:
    - dict: Monthly onboarding meeting requirements
    """
    monthly_onboarding = {}
    
    # Monthly meetings: 50% of new logos need 6 months of meetings
    monthly_customers = (new_logos_per_quarter * 
                        (onboarding_params['monthly_onboarding_percentage'] / 100))
    # 6 months of meetings spread across year = 6/12 = 0.5 factor
    monthly_onboarding['monthly_meetings'] = monthly_customers * 0.5
    
    # Quarterly meetings: 50% of new logos need 4 meetings per year
    quarterly_customers = (new_logos_per_quarter * 
                          (onboarding_params['quarterly_onboarding_percentage'] / 100))
    # 4 meetings per year = 4/12 meetings per month
    monthly_onboarding['quarterly_meetings'] = quarterly_customers * (4/12)
    
    return monthly_onboarding


def calculate_account_mgmt_hours(monthly_meetings, meeting_time_params):
    """
    Convert monthly meetings to weekly SE hours.
    
    Parameters:
    - monthly_meetings (dict): Monthly meeting counts by type
    - meeting_time_params (dict): Time per meeting including follow-up
    
    Returns:
    - float: Weekly hours for account management
    """
    total_monthly_meetings = sum(monthly_meetings.values())
    
    time_per_meeting = (meeting_time_params['meeting_duration_hours'] + 
                       meeting_time_params['meeting_followup_hours'])
    
    monthly_hours = total_monthly_meetings * time_per_meeting
    
    # Convert to weekly hours (assume 4.33 weeks per month)
    weekly_hours = monthly_hours / 4.33
    
    return weekly_hours


def calculate_strategic_workload(strategic_params, staffing_config):
    """
    Calculate strategic activity hours per role.
    
    Parameters:
    - strategic_params (dict): Strategic activity time parameters
    - staffing_config (dict): Team structure configuration
    
    Returns:
    - dict: Strategic hours by role type
    """
    # Calculate total strategic hours per week
    total_strategic_hours = 0
    
    # Handle min/max ranges by taking midpoint
    customer_zero_hours = (strategic_params['customer_zero_hours_min'] + 
                          strategic_params['customer_zero_hours_max']) / 2
    
    developer_advocacy_hours = (strategic_params['developer_advocacy_hours_min'] + 
                               strategic_params['developer_advocacy_hours_max']) / 2
    
    total_strategic_hours = (customer_zero_hours + 
                           strategic_params['content_creation_hours'] +
                           strategic_params['sales_enablement_hours'] +
                           developer_advocacy_hours +
                           strategic_params['asset_development_hours_avg'])
    
    return distribute_strategic_work(total_strategic_hours, 
                                   staffing_config['ic_strategic_percentage'],
                                   staffing_config['num_ic_ses'],
                                   staffing_config['num_directors'],
                                   staffing_config['director_ic_percentage'])


def distribute_strategic_work(total_hours, ic_percentage, num_ics, num_directors, director_ic_percentage):
    """
    Distribute strategic work between ICs and Directors.
    
    Parameters:
    - total_hours (float): Total strategic hours per week
    - ic_percentage (float): Percentage allocated to ICs
    - num_ics (int): Number of IC SEs
    - num_directors (int): Number of directors
    - director_ic_percentage (float): Percentage of director time on IC activities
    
    Returns:
    - dict: Strategic hours by role
    """
    ic_strategic_hours = total_hours * (ic_percentage / 100)
    director_strategic_hours = total_hours * ((100 - ic_percentage) / 100)
    
    # Calculate effective director capacity for strategic work
    # Directors spend (100 - director_ic_percentage)% on strategic activities
    director_strategic_capacity_percentage = 100 - director_ic_percentage
    
    return {
        'ic_strategic_hours_per_se': ic_strategic_hours / max(num_ics, 1),
        'director_strategic_hours_per_director': director_strategic_hours / max(num_directors, 1),
        'total_ic_strategic_hours': ic_strategic_hours,
        'total_director_strategic_hours': director_strategic_hours,
        'director_strategic_capacity_percentage': director_strategic_capacity_percentage
    }


def validate_percentages_sum_to_100(percentage_dict, tolerance=0.01):
    """
    Validate that percentages sum to 100%.
    
    Parameters:
    - percentage_dict (dict): Dictionary of percentage values
    - tolerance (float): Allowed deviation from 100%
    
    Returns:
    - bool: True if valid, False otherwise
    """
    total = sum(percentage_dict.values())
    return abs(total - 100) <= tolerance


def calculate_weekly_capacity(num_ses, hours_per_week=40):
    """
    Calculate total weekly capacity for SE team.
    
    Parameters:
    - num_ses (float): Number of SEs (can be fractional for part-time)
    - hours_per_week (float): Standard hours per week per SE
    
    Returns:
    - float: Total weekly capacity in hours
    """
    return num_ses * hours_per_week


def format_hours_for_display(hours):
    """
    Format hours for user-friendly display.
    
    Parameters:
    - hours (float): Hours to format
    
    Returns:
    - str: Formatted string
    """
    if hours < 0.1:
        return "< 0.1 hrs"
    elif hours < 1:
        return f"{hours:.1f} hrs"
    else:
        return f"{hours:.1f} hrs"


# Test helper functions with sample data
def test_helper_functions():
    """Test all helper functions with sample inputs to verify correctness."""
    print("🧪 Testing Helper Functions")
    print("=" * 40)
    
    test_results = []
    
    try:
        # Test revenue calculation
        result = calculate_required_new_logos(2000000, 75000)
        expected = 26.67
        assert abs(result - expected) < 0.1, f"Expected ~{expected}, got {result}"
        test_results.append("✅ calculate_required_new_logos: Working correctly")
        
        # Test win rate calculation
        eval_mix = {'self_guided_percentage': 65, 'se_led_percentage': 35}
        win_rates = {'self_guided_percentage': 35, 'se_led_percentage': 45}
        result = calculate_weighted_win_rate(eval_mix, win_rates)
        expected = 0.385  # (0.65 * 0.35) + (0.35 * 0.45)
        assert abs(result - expected) < 0.01, f"Expected ~{expected}, got {result}"
        test_results.append("✅ calculate_weighted_win_rate: Working correctly")
        
        # Test capacity calculation
        result = calculate_weekly_capacity(2.5, 40)
        expected = 100
        assert result == expected, f"Expected {expected}, got {result}"
        test_results.append("✅ calculate_weekly_capacity: Working correctly")
        
        # Test percentage validation
        valid_percentages = {'a': 30, 'b': 70}
        invalid_percentages = {'a': 30, 'b': 80}
        assert validate_percentages_sum_to_100(valid_percentages) == True
        assert validate_percentages_sum_to_100(invalid_percentages) == False
        test_results.append("✅ validate_percentages_sum_to_100: Working correctly")
        
        # Test hours formatting
        result = format_hours_for_display(12.567)
        expected = "12.6 hrs"
        assert result == expected, f"Expected {expected}, got {result}"
        test_results.append("✅ format_hours_for_display: Working correctly")
        
    except Exception as e:
        test_results.append(f"❌ Helper function test failed: {str(e)}")
    
    # Display results
    for result in test_results:
        print(result)
    
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print(f"\n📊 Test Summary: {success_count}/{total_count} tests passed")
    
    if success_count == total_count:
        print("🎉 All helper function tests passed!")
        return True
    else:
        print("⚠️  Some helper function tests failed.")
        return False

# Run tests
test_helper_functions()

print("\n✅ Helper Functions Framework implemented successfully")
print("📝 Ready for Step 3: Parameter Management")

---

# 2. Parameter Management {#parameter-management}

*Input parameter definitions, default values, and validation rules*

In [ ]:
# Comprehensive parameter management system with default values and validation

# ===================================
# PARAMETER DICTIONARIES WITH DEFAULTS
# ===================================

# Revenue and Business Target Parameters
revenue_params = {
    'quarterly_revenue_goal': 2000000,      # $2M quarterly revenue target
    'average_selling_price': 75000          # $75K average deal size
}

# Sales Conversion Funnel Parameters
conversion_params = {
    'opportunity_creation_rate': 50,        # 50% of meetings create opportunities
    'demo_conversion_rate': 99,             # 99% of opportunities get demos
    'tech_eval_conversion_rate': 99         # 99% of demos get technical evaluation
}

# Evaluation Mix Parameters (must sum to 100%)
eval_mix_params = {
    'self_guided_percentage': 65.5,         # Self-guided evaluations
    'se_led_percentage': 34.5,              # SE-led evaluations  
    'rapid_pov_percentage': 1.0,            # Rapid POV evaluations
    'no_eval_percentage': 0.0               # No evaluation deals
}

# Win Rate Parameters by Evaluation Type
win_rate_params = {
    'self_guided_win_rate': 35,             # 35% win rate for self-guided
    'se_led_win_rate': 45,                  # 45% win rate for SE-led
    'rapid_pov_win_rate': 60,               # 60% win rate for rapid POV
    'no_eval_win_rate': 10                  # 10% win rate for no evaluation
}

# Activity Time Requirements (hours)
activity_time_params = {
    'initial_meeting_time': 0.75,           # Initial meeting + prep time
    'demo_prep_time': 1.0,                  # Demo preparation time
    'demo_delivery_time': 1.0,              # Demo delivery time  
    'demo_followup_time': 0.75,             # Demo follow-up time
    'self_guided_support_hours_per_week': 2.0,     # Weekly support for self-guided
    'se_led_eval_hours_per_week': 4.0,             # Weekly hours for SE-led evals
    'rapid_pov_total_hours': 30             # Total hours for rapid POV
}

# Non-Pipeline Activity Hours (fixed weekly activities per SE)
fixed_activity_params = {
    'slack_email_hours': 5.0,               # Slack/email communication
    'troubleshooting_hours': 3.75,          # Technical troubleshooting
    'internal_coordination_hours': 7.75,    # Internal meetings/coordination
    'crm_admin_hours': 2.5                  # CRM and administrative tasks
}

# Team Structure and Staffing Parameters
staffing_params = {
    'num_ic_ses': 1,                        # Number of IC SEs
    'num_directors': 1,                     # Number of SE Directors
    'director_ic_percentage': 50            # % of director time on IC activities
}

# Strategic Activity Parameters (hours per week per FTE SE)
strategic_params = {
    'customer_zero_hours_min': 2.0,         # Customer zero minimum hours
    'customer_zero_hours_max': 4.0,         # Customer zero maximum hours
    'content_creation_hours': 2.0,          # Content creation hours
    'sales_enablement_hours': 1.0,          # Sales enablement hours
    'developer_advocacy_hours_min': 1.0,    # Developer advocacy minimum
    'developer_advocacy_hours_max': 2.0,    # Developer advocacy maximum
    'asset_development_hours_avg': 1.5,     # Asset development average hours
    'ic_strategic_percentage': 70           # % of strategic work for ICs vs Directors
}

# Account Portfolio Parameters
account_params = {
    'strategic_accounts': 3,                # Number of strategic accounts
    'protect_accounts': 7,                  # Number of protect accounts
    'growth_accounts': 24,                  # Number of growth accounts
    'retain_accounts': 59                   # Number of retain accounts
}

# Meeting Frequency Parameters
meeting_frequency_params = {
    'strategic_meetings_per_month': 1.0,    # Strategic account meetings/month
    'protect_meetings_per_month': 1.0,      # Protect account meetings/month
    'growth_meetings_per_month': 1.0,       # Growth account meetings/month
    'retain_meetings_per_months': 9         # Retain accounts: every X months
}

# New Customer Onboarding Parameters
onboarding_params = {
    'new_logos_per_quarter': 7,             # Expected new logos per quarter
    'monthly_onboarding_percentage': 50,    # % needing monthly meetings (6 months)
    'quarterly_onboarding_percentage': 50   # % needing quarterly meetings (4/year)
}

# Meeting Time Investment Parameters
meeting_time_params = {
    'meeting_duration_hours': 0.5,          # Base meeting duration
    'meeting_followup_hours': 0.625         # Follow-up time per meeting
}

# ===================================
# PARAMETER VALIDATION FUNCTIONS
# ===================================

def validate_eval_mix_percentages():
    """Validate that evaluation mix percentages sum to 100%."""
    total = sum(eval_mix_params.values())
    tolerance = 0.01  # Allow small floating point errors
    
    if abs(total - 100) > tolerance:
        return False, f\"Evaluation mix percentages sum to {total:.1f}%, must equal 100%\"
    return True, \"Evaluation mix percentages valid\"

def validate_onboarding_percentages():
    """Validate that onboarding percentages sum to ≤100%."""
    total = (onboarding_params['monthly_onboarding_percentage'] + 
             onboarding_params['quarterly_onboarding_percentage'])
    
    if total > 100:
        return False, f\"Onboarding percentages sum to {total}%, must be ≤100%\"
    return True, \"Onboarding percentages valid\"

def validate_positive_values():
    """Validate that all time and count parameters are positive."""
    
    # Check all parameter dictionaries for positive values
    param_groups = [
        ('revenue_params', revenue_params),
        ('conversion_params', conversion_params),
        ('activity_time_params', activity_time_params),
        ('fixed_activity_params', fixed_activity_params),
        ('staffing_params', staffing_params),
        ('strategic_params', strategic_params),
        ('account_params', account_params),
        ('meeting_frequency_params', meeting_frequency_params),
        ('meeting_time_params', meeting_time_params)
    ]
    
    errors = []
    
    for group_name, params in param_groups:
        for param_name, value in params.items():
            if value < 0:
                errors.append(f\"{group_name}.{param_name}: {value} (must be ≥ 0)\")
    
    if errors:
        return False, f\"Negative values found: {', '.join(errors)}\"
    return True, \"All values are positive\"

def validate_percentage_bounds():
    """Validate that all percentage parameters are between 0-100%."""
    
    # Percentage parameters to check
    percentage_params = [
        ('conversion_params.opportunity_creation_rate', conversion_params['opportunity_creation_rate']),
        ('conversion_params.demo_conversion_rate', conversion_params['demo_conversion_rate']),
        ('conversion_params.tech_eval_conversion_rate', conversion_params['tech_eval_conversion_rate']),
        ('eval_mix_params.self_guided_percentage', eval_mix_params['self_guided_percentage']),
        ('eval_mix_params.se_led_percentage', eval_mix_params['se_led_percentage']),
        ('eval_mix_params.rapid_pov_percentage', eval_mix_params['rapid_pov_percentage']),
        ('eval_mix_params.no_eval_percentage', eval_mix_params['no_eval_percentage']),
        ('win_rate_params.self_guided_win_rate', win_rate_params['self_guided_win_rate']),
        ('win_rate_params.se_led_win_rate', win_rate_params['se_led_win_rate']),
        ('win_rate_params.rapid_pov_win_rate', win_rate_params['rapid_pov_win_rate']),
        ('win_rate_params.no_eval_win_rate', win_rate_params['no_eval_win_rate']),
        ('staffing_params.director_ic_percentage', staffing_params['director_ic_percentage']),
        ('strategic_params.ic_strategic_percentage', strategic_params['ic_strategic_percentage']),
        ('onboarding_params.monthly_onboarding_percentage', onboarding_params['monthly_onboarding_percentage']),
        ('onboarding_params.quarterly_onboarding_percentage', onboarding_params['quarterly_onboarding_percentage'])
    ]
    
    errors = []
    
    for param_name, value in percentage_params:
        if not (0 <= value <= 100):
            errors.append(f\"{param_name}: {value}% (must be 0-100%)\")
    
    if errors:
        return False, f\"Invalid percentage values: {', '.join(errors)}\"
    return True, \"All percentage values are valid\"

def validate_logical_constraints():
    """Validate logical business constraints."""
    warnings = []
    errors = []
    
    # Check for unrealistic conversion rates
    if conversion_params['opportunity_creation_rate'] > 80:
        warnings.append(f\"High opportunity creation rate: {conversion_params['opportunity_creation_rate']}%\")
    
    # Check for unrealistic win rates
    avg_win_rate = sum(win_rate_params.values()) / len(win_rate_params)
    if avg_win_rate > 60:
        warnings.append(f\"High average win rate: {avg_win_rate:.1f}%\")
    
    # Check for unrealistic team structure
    total_directors = staffing_params['num_directors']
    total_ics = staffing_params['num_ic_ses']
    if total_directors > total_ics and total_ics > 0:
        warnings.append(f\"More directors ({total_directors}) than ICs ({total_ics})\")
    
    # Check for account load reasonableness
    total_accounts = sum(account_params.values())
    total_se_capacity = total_ics + total_directors
    if total_accounts > 0 and total_se_capacity > 0:
        accounts_per_se = total_accounts / total_se_capacity
        if accounts_per_se > 50:
            warnings.append(f\"High account load: {accounts_per_se:.1f} accounts per SE\")
    
    return errors, warnings

# ===================================
# CENTRAL PARAMETER UPDATE FUNCTION
# ===================================

def update_all_parameters():
    \"\"\"Apply comprehensive validation and return validation results.\"\"\"
    print(\"🔍 Validating Parameters\")
    print(\"=\" * 30)
    
    validation_results = []
    warnings = []
    
    # Run all validation checks
    validations = [
        validate_eval_mix_percentages(),
        validate_onboarding_percentages(),
        validate_positive_values(),
        validate_percentage_bounds()
    ]
    
    # Check validation results
    all_valid = True
    for is_valid, message in validations:
        if is_valid:
            validation_results.append(f\"✅ {message}\")
        else:
            validation_results.append(f\"❌ {message}\")
            all_valid = False
    
    # Check logical constraints
    errors, constraint_warnings = validate_logical_constraints()
    
    for error in errors:
        validation_results.append(f\"❌ {error}\")
        all_valid = False
    
    warnings.extend(constraint_warnings)
    
    # Display results
    for result in validation_results:
        print(result)
    
    if warnings:
        print(\"\\n⚠️  Warnings:\")
        for warning in warnings:
            print(f\"   • {warning}\")
    
    print(f\"\\n📊 Validation Summary:\")
    if all_valid:
        print(\"✅ All parameters valid and ready for calculations\")
        if warnings:
            print(f\"⚠️  {len(warnings)} warnings (calculations will proceed)\")
    else:
        print(\"❌ Parameter validation failed - please correct errors\")
    
    return all_valid, warnings

# ===================================
# PARAMETER SUMMARY DISPLAY
# ===================================

def display_parameter_summary():
    \"\"\"Display a comprehensive summary of all current parameters.\"\"\"
    print(\"📋 Parameter Summary\")
    print(\"=\" * 50)
    
    print(\"\\n💰 Revenue & Business Targets:\")
    print(f\"   • Quarterly Revenue Goal: ${revenue_params['quarterly_revenue_goal']:,}\")
    print(f\"   • Average Selling Price: ${revenue_params['average_selling_price']:,}\")
    
    print(\"\\n🔄 Sales Conversion Funnel:\")
    for param, value in conversion_params.items():
        print(f\"   • {param.replace('_', ' ').title()}: {value}%\")
    
    print(\"\\n📊 Evaluation Mix:\")
    for param, value in eval_mix_params.items():
        print(f\"   • {param.replace('_', ' ').title()}: {value}%\")
    
    print(\"\\n🎯 Win Rates by Evaluation Type:\")
    for param, value in win_rate_params.items():
        print(f\"   • {param.replace('_', ' ').title()}: {value}%\")
    
    print(\"\\n⏰ Activity Time Requirements:\")
    for param, value in activity_time_params.items():
        print(f\"   • {param.replace('_', ' ').title()}: {value} hrs\")
    
    print(\"\\n👥 Team Structure:\")
    print(f\"   • IC SEs: {staffing_params['num_ic_ses']}\")
    print(f\"   • Directors: {staffing_params['num_directors']}\")
    print(f\"   • Director IC Time: {staffing_params['director_ic_percentage']}%\")
    
    print(\"\\n🏢 Account Portfolio:\")
    total_accounts = sum(account_params.values())
    for param, value in account_params.items():
        percentage = (value / total_accounts * 100) if total_accounts > 0 else 0
        print(f\"   • {param.replace('_', ' ').title()}: {value} ({percentage:.1f}%)\")
    
    print(f\"\\n📈 Strategic Activities (IC Focus: {strategic_params['ic_strategic_percentage']}%):\")
    print(f\"   • Customer Zero: {strategic_params['customer_zero_hours_min']}-{strategic_params['customer_zero_hours_max']} hrs/week\")
    print(f\"   • Content Creation: {strategic_params['content_creation_hours']} hrs/week\")
    print(f\"   • Sales Enablement: {strategic_params['sales_enablement_hours']} hrs/week\")
    print(f\"   • Developer Advocacy: {strategic_params['developer_advocacy_hours_min']}-{strategic_params['developer_advocacy_hours_max']} hrs/week\")
    print(f\"   • Asset Development: {strategic_params['asset_development_hours_avg']} hrs/week (avg)\")

# ===================================
# PARAMETER TESTING
# ===================================

def test_parameter_management():
    \"\"\"Test parameter management system with various scenarios.\"\"\"
    print(\"🧪 Testing Parameter Management System\")
    print(\"=\" * 45)
    
    test_results = []
    
    try:
        # Test 1: Default parameters should be valid
        is_valid, warnings = update_all_parameters()
        if is_valid:
            test_results.append(\"✅ Default parameters: Valid\")
        else:
            test_results.append(\"❌ Default parameters: Invalid\")
        
        # Test 2: Test evaluation mix validation
        original_mix = eval_mix_params.copy()
        eval_mix_params['self_guided_percentage'] = 50
        eval_mix_params['se_led_percentage'] = 60  # This should make total > 100%
        
        is_valid, message = validate_eval_mix_percentages()
        if not is_valid:
            test_results.append(\"✅ Evaluation mix validation: Correctly detects invalid sum\")
        else:
            test_results.append(\"❌ Evaluation mix validation: Failed to detect invalid sum\")
        
        # Restore original values
        eval_mix_params.update(original_mix)
        
        # Test 3: Test percentage bounds validation
        original_conversion = conversion_params['opportunity_creation_rate']
        conversion_params['opportunity_creation_rate'] = 150  # Invalid percentage
        
        is_valid, message = validate_percentage_bounds()
        if not is_valid:
            test_results.append(\"✅ Percentage bounds validation: Correctly detects out-of-bounds\")
        else:
            test_results.append(\"❌ Percentage bounds validation: Failed to detect out-of-bounds\")
        
        # Restore original value
        conversion_params['opportunity_creation_rate'] = original_conversion
        
        # Test 4: Test positive values validation
        original_revenue = revenue_params['quarterly_revenue_goal']
        revenue_params['quarterly_revenue_goal'] = -1000000  # Invalid negative
        
        is_valid, message = validate_positive_values()
        if not is_valid:
            test_results.append(\"✅ Positive values validation: Correctly detects negative values\")
        else:
            test_results.append(\"❌ Positive values validation: Failed to detect negative values\")
        
        # Restore original value
        revenue_params['quarterly_revenue_goal'] = original_revenue
        
    except Exception as e:
        test_results.append(f\"❌ Parameter management test failed: {str(e)}\")
    
    # Display test results
    print(\"\\n📊 Test Results:\")
    for result in test_results:
        print(result)
    
    success_count = sum(1 for result in test_results if result.startswith(\"✅\"))
    total_count = len(test_results)
    
    print(f\"\\n📈 Test Summary: {success_count}/{total_count} tests passed\")
    
    if success_count == total_count:
        print(\"🎉 All parameter management tests passed!\")
        return True
    else:
        print(\"⚠️  Some parameter management tests failed.\")
        return False

# ===================================
# INITIALIZATION
# ===================================

# Run initial validation and display
print(\"🔧 Parameter Management System Initialized\")
print(\"=\" * 50)

# Test the parameter management system
test_parameter_management()

print(\"\\n\" + \"=\" * 50)
# Validate current parameters
is_valid, warnings = update_all_parameters()

if is_valid:
    print(\"\\n📋 Current Parameter Configuration:\")
    display_parameter_summary()
    print(\"\\n✅ Parameter Management System ready for use\")
    print(\"📝 Ready for Step 4: Revenue & Pipeline Calculations\")
else:
    print(\"\\n❌ Please correct parameter validation errors before proceeding\")
    print(\"📝 Fix errors and re-run parameter validation\")"

---

# 3. Revenue & Pipeline Calculations {#revenue-pipeline}

*Core funnel math and workload calculations driven by revenue targets*

In [ ]:
# Revenue & Pipeline Calculation Engine
# Core business logic for revenue-driven workload modeling

# ===================================
# PIPELINE CALCULATION ENGINE
# ===================================

def run_pipeline_calculations():
    """
    Execute complete pipeline calculation flow from revenue targets to SE workload.
    
    Returns:
    - dict: Comprehensive pipeline analysis results
    """
    print("💰 Running Revenue & Pipeline Calculations")
    print("=" * 50)
    
    # Step 1: Revenue-to-Activity Calculation
    print("\\n📊 Step 1: Revenue Target Analysis")
    print("-" * 35)
    
    # Calculate required new logos
    new_logos_needed = calculate_required_new_logos(
        revenue_params['quarterly_revenue_goal'],
        revenue_params['average_selling_price']
    )
    
    print(f"💰 Revenue Goal: ${revenue_params['quarterly_revenue_goal']:,}/quarter")
    print(f"💵 Average Deal Size: ${revenue_params['average_selling_price']:,}")
    print(f"🎯 Required New Logos: {new_logos_needed:.1f}/quarter ({new_logos_needed/13:.1f}/week)")
    
    # Step 2: Pipeline Funnel Calculations
    print("\\n🔄 Step 2: Pipeline Funnel Analysis")
    print("-" * 38)
    
    # Calculate weighted win rate
    weighted_win_rate = calculate_weighted_win_rate(eval_mix_params, win_rate_params)
    
    print(f"📈 Weighted Win Rate: {weighted_win_rate:.1%}")
    print("\\n🎲 Win Rate Breakdown:")
    for eval_type, percentage in eval_mix_params.items():
        win_rate_key = eval_type.replace('_percentage', '_win_rate')
        if win_rate_key in win_rate_params and percentage > 0:
            print(f"   • {eval_type.replace('_', ' ').title()}: {percentage:.1f}% of mix @ {win_rate_params[win_rate_key]}% win rate")
    
    # Calculate required activities using helper function
    weekly_activities = calculate_required_activities(
        new_logos_needed, 
        conversion_params, 
        eval_mix_params, 
        win_rate_params
    )
    
    print("\\n📅 Required Weekly Activities:")
    print(f"   • Initial Meetings: {weekly_activities['initial_meetings']:.1f}/week")
    print(f"   • Demos: {weekly_activities['demos']:.1f}/week")
    print(f"   • Total Evaluations: {weekly_activities['total_evaluations']:.1f}/week")
    
    # Step 3: Evaluation Workload Breakdown
    print("\\n🔬 Step 3: Evaluation Workload Analysis")
    print("-" * 42)
    
    # Break down evaluations by type
    evaluation_breakdown = {}
    for eval_type, percentage in eval_mix_params.items():
        if percentage > 0:
            eval_key = eval_type.replace('_percentage', '')
            count = weekly_activities['total_evaluations'] * (percentage / 100)
            evaluation_breakdown[eval_key] = count
            print(f"   • {eval_type.replace('_', ' ').title()}: {count:.1f}/week")
    
    # Calculate time requirements for each evaluation type
    evaluation_hours = calculate_evaluation_workload(evaluation_breakdown, activity_time_params)
    
    print("\\n⏰ Evaluation Time Requirements (per week):")
    total_eval_hours = 0
    for eval_type, hours in evaluation_hours.items():
        if hours > 0:
            print(f"   • {eval_type.replace('_', ' ').title()}: {hours:.1f} hrs")
            total_eval_hours += hours
    print(f"   • Total Evaluation Hours: {total_eval_hours:.1f} hrs/week")
    
    # Step 4: Demo and Meeting Workload
    print("\\n🎤 Step 4: Demo & Meeting Workload Analysis")
    print("-" * 45)
    
    # Calculate demo workload
    demo_hours = calculate_demo_workload(weekly_activities['demos'], activity_time_params)
    print(f"🎯 Demo Workload: {demo_hours:.1f} hrs/week")
    print(f"   • Prep: {weekly_activities['demos'] * activity_time_params['demo_prep_time']:.1f} hrs")
    print(f"   • Delivery: {weekly_activities['demos'] * activity_time_params['demo_delivery_time']:.1f} hrs")
    print(f"   • Follow-up: {weekly_activities['demos'] * activity_time_params['demo_followup_time']:.1f} hrs")
    
    # Calculate meeting workload
    meeting_hours = calculate_meeting_workload(
        weekly_activities['initial_meetings'], 
        activity_time_params['initial_meeting_time']
    )
    print(f"🤝 Initial Meetings: {meeting_hours:.1f} hrs/week ({weekly_activities['initial_meetings']:.1f} meetings)")
    
    # Step 5: Pipeline Activity Summary
    print("\\n📋 Step 5: Pipeline Activity Summary")
    print("-" * 37)
    
    total_pipeline_hours = total_eval_hours + demo_hours + meeting_hours
    
    pipeline_summary = {
        'revenue_goal': revenue_params['quarterly_revenue_goal'],
        'new_logos_needed': new_logos_needed,
        'weighted_win_rate': weighted_win_rate,
        'weekly_activities': weekly_activities,
        'evaluation_breakdown': evaluation_breakdown,
        'evaluation_hours': evaluation_hours,
        'demo_hours': demo_hours,
        'meeting_hours': meeting_hours,
        'total_pipeline_hours': total_pipeline_hours
    }
    
    print(f"💼 Total Pipeline Hours: {total_pipeline_hours:.1f} hrs/week")
    print(f"   • Meetings: {meeting_hours:.1f} hrs ({meeting_hours/total_pipeline_hours:.1%})")
    print(f"   • Demos: {demo_hours:.1f} hrs ({demo_hours/total_pipeline_hours:.1%})")
    print(f"   • Evaluations: {total_eval_hours:.1f} hrs ({total_eval_hours/total_pipeline_hours:.1%})")
    
    return pipeline_summary

def calculate_pipeline_efficiency_metrics(pipeline_results):
    """
    Calculate efficiency and performance metrics for the pipeline.
    
    Parameters:
    - pipeline_results (dict): Results from run_pipeline_calculations()
    
    Returns:
    - dict: Efficiency metrics and insights
    """
    
    print("\\n📊 Pipeline Efficiency Analysis")
    print("=" * 35)
    
    # Calculate key ratios
    weekly_activities = pipeline_results['weekly_activities']
    
    # Conversion efficiency through funnel
    conversion_efficiency = {
        'meetings_to_opportunities': conversion_params['opportunity_creation_rate'] / 100,
        'opportunities_to_demos': conversion_params['demo_conversion_rate'] / 100,
        'demos_to_evaluations': conversion_params['tech_eval_conversion_rate'] / 100,
        'evaluations_to_wins': pipeline_results['weighted_win_rate']
    }
    
    # Overall funnel efficiency (meetings to wins)
    overall_efficiency = 1
    for stage, rate in conversion_efficiency.items():
        overall_efficiency *= rate
    
    print(f"🎯 Overall Funnel Efficiency: {overall_efficiency:.1%}")
    print(f"   • Meetings → Opportunities: {conversion_efficiency['meetings_to_opportunities']:.1%}")
    print(f"   • Opportunities → Demos: {conversion_efficiency['opportunities_to_demos']:.1%}")
    print(f"   • Demos → Evaluations: {conversion_efficiency['demos_to_evaluations']:.1%}")
    print(f"   • Evaluations → Wins: {conversion_efficiency['evaluations_to_wins']:.1%}")
    
    # Time investment per new logo
    time_per_new_logo = pipeline_results['total_pipeline_hours'] / (pipeline_results['new_logos_needed'] / 13)
    
    print(f"\\n⏱️  Time Investment Metrics:")
    print(f"   • Hours per New Logo: {time_per_new_logo:.1f} hrs")
    print(f"   • Hours per $1M Revenue: {pipeline_results['total_pipeline_hours'] / (revenue_params['quarterly_revenue_goal'] / 1000000 / 13):.1f} hrs/week")
    
    # Activity intensity metrics
    print(f"\\n📈 Activity Intensity:")
    print(f"   • Meetings per Day: {weekly_activities['initial_meetings'] / 5:.1f}")
    print(f"   • Demos per Day: {weekly_activities['demos'] / 5:.1f}")
    print(f"   • Evaluations per Day: {weekly_activities['total_evaluations'] / 5:.1f}")
    
    efficiency_metrics = {
        'conversion_efficiency': conversion_efficiency,
        'overall_efficiency': overall_efficiency,
        'time_per_new_logo': time_per_new_logo,
        'hours_per_million_revenue': pipeline_results['total_pipeline_hours'] / (revenue_params['quarterly_revenue_goal'] / 1000000 / 13)
    }
    
    return efficiency_metrics

def create_pipeline_dataframe(pipeline_results):
    """
    Create a comprehensive DataFrame summarizing pipeline calculations.
    
    Parameters:
    - pipeline_results (dict): Results from run_pipeline_calculations()
    
    Returns:
    - pd.DataFrame: Structured pipeline data
    """
    
    # Create pipeline summary table
    pipeline_data = []
    
    # Revenue and target data
    pipeline_data.append({
        'Category': 'Revenue Targets',
        'Metric': 'Quarterly Revenue Goal',
        'Weekly Value': revenue_params['quarterly_revenue_goal'] / 13,
        'Quarterly Value': revenue_params['quarterly_revenue_goal'],
        'Unit': '$',
        'Notes': f'${revenue_params["quarterly_revenue_goal"]:,} target'
    })
    
    pipeline_data.append({
        'Category': 'Revenue Targets', 
        'Metric': 'Required New Logos',
        'Weekly Value': pipeline_results['new_logos_needed'] / 13,
        'Quarterly Value': pipeline_results['new_logos_needed'],
        'Unit': 'logos',
        'Notes': f'${revenue_params["average_selling_price"]:,} avg deal size'
    })
    
    # Activity requirements
    for activity, count in pipeline_results['weekly_activities'].items():
        if activity != 'total_evaluations':  # Skip total since we have breakdown
            pipeline_data.append({
                'Category': 'Required Activities',
                'Metric': activity.replace('_', ' ').title(),
                'Weekly Value': count,
                'Quarterly Value': count * 13,
                'Unit': 'activities',
                'Notes': f'{count:.1f} per week required'
            })
    
    # Time requirements
    time_data = [
        ('Meeting Hours', pipeline_results['meeting_hours'], 'meetings'),
        ('Demo Hours', pipeline_results['demo_hours'], 'demos'),
        ('Total Evaluation Hours', sum(pipeline_results['evaluation_hours'].values()), 'evaluations')
    ]
    
    for metric, hours, source in time_data:
        pipeline_data.append({
            'Category': 'Time Requirements',
            'Metric': metric,
            'Weekly Value': hours,
            'Quarterly Value': hours * 13,
            'Unit': 'hours',
            'Notes': f'From {source}'
        })
    
    # Total pipeline workload
    pipeline_data.append({
        'Category': 'Total Workload',
        'Metric': 'Total Pipeline Hours',
        'Weekly Value': pipeline_results['total_pipeline_hours'],
        'Quarterly Value': pipeline_results['total_pipeline_hours'] * 13,
        'Unit': 'hours',
        'Notes': 'All pipeline activities combined'
    })
    
    df = pd.DataFrame(pipeline_data)
    
    print("\\n📊 Pipeline Summary Table")
    print("=" * 30)
    
    # Display formatted table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    
    # Format numeric columns
    display_df = df.copy()
    display_df['Weekly Value'] = display_df['Weekly Value'].apply(lambda x: f'{x:.1f}')
    display_df['Quarterly Value'] = display_df['Quarterly Value'].apply(lambda x: f'{x:.1f}')
    
    print(display_df.to_string(index=False))
    
    return df

def validate_pipeline_calculations(pipeline_results):
    """
    Validate pipeline calculations for reasonableness and logical consistency.
    
    Parameters:
    - pipeline_results (dict): Results from run_pipeline_calculations()
    
    Returns:
    - tuple: (is_valid, validation_messages)
    """
    
    print("\\n🔍 Pipeline Validation")
    print("=" * 25)
    
    validation_messages = []
    warnings = []
    is_valid = True
    
    # Check for reasonable activity levels
    weekly_activities = pipeline_results['weekly_activities']
    
    # Validate meeting frequency (should be realistic for SE capacity)
    if weekly_activities['initial_meetings'] > 50:
        warnings.append(f\"High meeting load: {weekly_activities['initial_meetings']:.1f} meetings/week\")
    
    if weekly_activities['demos'] > 25:
        warnings.append(f\"High demo load: {weekly_activities['demos']:.1f} demos/week\")
    
    # Check total hours against reasonable SE capacity
    total_hours = pipeline_results['total_pipeline_hours']
    if total_hours > 40:
        warnings.append(f\"Pipeline hours exceed full-time capacity: {total_hours:.1f} hrs/week\")
    
    # Validate win rate reasonableness
    if pipeline_results['weighted_win_rate'] > 0.7:
        warnings.append(f\"Very high win rate: {pipeline_results['weighted_win_rate']:.1%}\")
    elif pipeline_results['weighted_win_rate'] < 0.2:
        warnings.append(f\"Very low win rate: {pipeline_results['weighted_win_rate']:.1%}\")
    
    # Check for mathematical consistency
    expected_total_evals = sum(pipeline_results['evaluation_breakdown'].values())
    actual_total_evals = weekly_activities['total_evaluations']
    
    if abs(expected_total_evals - actual_total_evals) > 0.01:
        validation_messages.append(f\"❌ Evaluation breakdown inconsistency: {expected_total_evals:.2f} vs {actual_total_evals:.2f}\")
        is_valid = False
    else:
        validation_messages.append(\"✅ Evaluation breakdown mathematically consistent\")
    
    # Validate conversion rates produce expected results
    meetings = weekly_activities['initial_meetings']
    opportunities = meetings * (conversion_params['opportunity_creation_rate'] / 100)
    expected_demos = opportunities * (conversion_params['demo_conversion_rate'] / 100)
    
    if abs(expected_demos - weekly_activities['demos']) > 0.01:
        validation_messages.append(f\"❌ Demo calculation inconsistency\")
        is_valid = False
    else:
        validation_messages.append(\"✅ Funnel calculations mathematically consistent\")
    
    # Display results
    for message in validation_messages:
        print(message)
    
    if warnings:
        print(\"\\n⚠️  Warnings:\")
        for warning in warnings:
            print(f\"   • {warning}\")
    
    print(f\"\\n📊 Validation Summary:\")
    if is_valid:
        print(\"✅ Pipeline calculations validated successfully\")
        if warnings:
            print(f\"⚠️  {len(warnings)} warnings noted\")
    else:
        print(\"❌ Pipeline validation failed - check calculations\")
    
    return is_valid, validation_messages + warnings

# ===================================
# SCENARIO ANALYSIS FUNCTIONS
# ===================================

def run_sensitivity_analysis():
    """Run sensitivity analysis on key pipeline parameters."""
    
    print("\\n🎭 Pipeline Sensitivity Analysis")
    print("=" * 35)
    
    base_results = run_pipeline_calculations()
    base_hours = base_results['total_pipeline_hours']
    
    print(f\"\\n📊 Base Case: {base_hours:.1f} hrs/week\")
    
    # Test sensitivity to key parameters
    sensitivities = []
    
    # Revenue goal sensitivity
    original_revenue = revenue_params['quarterly_revenue_goal']
    for multiplier in [0.5, 0.75, 1.25, 1.5]:
        revenue_params['quarterly_revenue_goal'] = original_revenue * multiplier
        test_results = run_pipeline_calculations()
        change = (test_results['total_pipeline_hours'] - base_hours) / base_hours
        sensitivities.append({
            'Parameter': 'Revenue Goal',
            'Change': f'{multiplier:.0%} of base',
            'Hours': test_results['total_pipeline_hours'],
            'Change %': f'{change:+.1%}'
        })
    revenue_params['quarterly_revenue_goal'] = original_revenue
    
    # Win rate sensitivity  
    original_win_rates = win_rate_params.copy()
    for multiplier in [0.8, 0.9, 1.1, 1.2]:
        for rate_key in win_rate_params:
            win_rate_params[rate_key] = min(100, original_win_rates[rate_key] * multiplier)
        test_results = run_pipeline_calculations()
        change = (test_results['total_pipeline_hours'] - base_hours) / base_hours
        sensitivities.append({
            'Parameter': 'Win Rates',
            'Change': f'{multiplier:.0%} of base',
            'Hours': test_results['total_pipeline_hours'],
            'Change %': f'{change:+.1%}'
        })
    win_rate_params.update(original_win_rates)
    
    # Display sensitivity results
    sensitivity_df = pd.DataFrame(sensitivities)
    print(\"\\n📈 Sensitivity Results:\")
    print(sensitivity_df.to_string(index=False))
    
    return sensitivity_df

# ===================================
# MAIN EXECUTION
# ===================================

# Run the complete pipeline calculation flow
pipeline_results = run_pipeline_calculations()

# Calculate efficiency metrics
efficiency_metrics = calculate_pipeline_efficiency_metrics(pipeline_results)

# Create summary DataFrame
pipeline_df = create_pipeline_dataframe(pipeline_results)

# Validate calculations
is_valid, validation_messages = validate_pipeline_calculations(pipeline_results)

# Store results for use in other sections
PIPELINE_RESULTS = {
    'pipeline_summary': pipeline_results,
    'efficiency_metrics': efficiency_metrics,
    'pipeline_df': pipeline_df,
    'validation_status': is_valid,
    'validation_messages': validation_messages
}

if is_valid:
    print(\"\\n✅ Revenue & Pipeline Calculations completed successfully\")
    print(\"📝 Ready for Step 5: Account Management Logic\")
    
    # Run sensitivity analysis
    sensitivity_results = run_sensitivity_analysis()
    PIPELINE_RESULTS['sensitivity_analysis'] = sensitivity_results
    
else:
    print(\"\\n❌ Pipeline calculations need review before proceeding\")
    print(\"📝 Please address validation issues\")"

---

# 4. Account Management Logic {#account-management}

*Meeting frequency calculations and time allocation for existing customers*

In [ ]:
# Placeholder for account management logic
# Will be implemented in Step 5

print("👥 Account management logic section ready for implementation")

---

# 5. Strategic Activities Logic {#strategic-activities}

*Strategic time allocation and distribution between ICs and Directors*

In [ ]:
# Placeholder for strategic activities logic
# Will be implemented in Step 6

print("🎯 Strategic activities logic section ready for implementation")

---

# 6. Interactive Parameter Controls {#interactive-controls}

*Widget-based interface for parameter input and real-time updates*

In [ ]:
# Placeholder for interactive controls
# Will be implemented in Steps 7-9

print("🎛️ Interactive controls section ready for implementation")

---

# 7. Real-time Calculations {#real-time-calculations}

*Live calculation updates based on parameter changes*

In [ ]:
# Placeholder for real-time calculations
# Will be implemented in Step 9

print("⚡ Real-time calculations section ready for implementation")

---

# 8. Core Visualizations {#core-visualizations}

*Essential charts for utilization analysis and capacity planning*

In [ ]:
# Placeholder for core visualizations
# Will be implemented in Step 10

print("📊 Core visualizations section ready for implementation")

---

# 9. Executive Dashboard {#executive-dashboard}

*Professional dashboard with comprehensive business insights*

In [ ]:
# Placeholder for executive dashboard
# Will be implemented in Step 11

print("📈 Executive dashboard section ready for implementation")

---

# 10. Results Summary & Export {#results-summary}

*Executive summary generation and export capabilities*

In [ ]:
# Placeholder for results summary and export
# Will be implemented in Step 12

print("📋 Results summary & export section ready for implementation")

---

# 11. Testing Framework {#testing}

*Comprehensive validation and testing of all calculations*

In [ ]:
# Placeholder for testing framework
# Will be implemented throughout all steps

print("🧪 Testing framework section ready for implementation")

---

# 12. Usage Examples {#examples}

*Sample scenarios and practical use cases*

In [ ]:
# Placeholder for usage examples
# Will be implemented in final step

print("💡 Usage examples section ready for implementation")

---

## Import Verification Test

*Basic test to verify all imports are working correctly*

In [ ]:
# Test all imported libraries
def test_imports():
    """Verify all required libraries are properly imported and functional."""
    test_results = []
    
    # Test pandas
    try:
        test_df = pd.DataFrame({'test': [1, 2, 3]})
        assert len(test_df) == 3
        test_results.append("✅ pandas: Working correctly")
    except Exception as e:
        test_results.append(f"❌ pandas: Error - {str(e)}")
    
    # Test numpy
    try:
        test_array = np.array([1, 2, 3])
        assert np.sum(test_array) == 6
        test_results.append("✅ numpy: Working correctly")
    except Exception as e:
        test_results.append(f"❌ numpy: Error - {str(e)}")
    
    # Test matplotlib
    try:
        fig, ax = plt.subplots(1, 1, figsize=(4, 3))
        ax.plot([1, 2, 3], [1, 4, 2])
        plt.close(fig)
        test_results.append("✅ matplotlib: Working correctly")
    except Exception as e:
        test_results.append(f"❌ matplotlib: Error - {str(e)}")
    
    # Test seaborn
    try:
        current_style = sns.axes_style()
        assert isinstance(current_style, dict)
        test_results.append("✅ seaborn: Working correctly")
    except Exception as e:
        test_results.append(f"❌ seaborn: Error - {str(e)}")
    
    # Test ipywidgets
    try:
        test_widget = widgets.IntSlider(value=5, min=0, max=10)
        assert test_widget.value == 5
        test_results.append("✅ ipywidgets: Working correctly")
    except Exception as e:
        test_results.append(f"❌ ipywidgets: Error - {str(e)}")
    
    # Display results
    print("\n🔍 Import Verification Results:")
    print("=" * 40)
    for result in test_results:
        print(result)
    
    # Summary
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print("\n📊 Summary:")
    print(f"   • {success_count}/{total_count} libraries working correctly")
    
    if success_count == total_count:
        print("\n🎉 All imports successful! Ready to proceed with implementation.")
        return True
    else:
        print("\n⚠️  Some imports failed. Please check your environment.")
        return False

# Run the test
test_imports()

---

## Next Steps

✅ **Step 1 Complete**: Basic notebook structure created with all required sections

### Ready for Implementation:
- **Step 2**: Helper Functions Framework - Core calculation functions
- **Step 3**: Parameter Management - Input parameter definitions and validation

### Implementation Notes:
- All libraries imported and tested successfully
- Professional styling and configuration applied
- Clear section structure with table of contents navigation
- Placeholder cells ready for modular development
- Testing framework initialized

**📝 Continue with Step 2 to implement the helper functions framework.**